## Cel·la de prova

ha de sortir una ruta que inclogui envs/myenv

In [1]:
import sys
print(sys.executable)

c:\Users\UX433\miniconda3\envs\myenv\python.exe


# Crear blueprint

### Comprobació preliminar del directori i kernell

Per verificar que datasets.ods i el notebook estan a la llista

In [2]:
import os, sys
print("Python executable:", sys.executable)     # mostra el Python que corre (ha de ser el env activat)
print("Directori de treball:", os.getcwd())    # carpeta del repo
print("Llistat d'arxius a la carpeta:")
for f in sorted(os.listdir('.')):
    print(" -", f)

Python executable: c:\Users\UX433\miniconda3\envs\myenv\python.exe
Directori de treball: c:\Users\UX433\OneDrive\Escritorio\networkalignment
Llistat d'arxius a la carpeta:
 - .git
 - C-el_k4_Synthetic- No groups - Simoultenously.ipynb
 - crear_blueprint.ipynb
 - datasets.ods
 - environment.yml


In [3]:
import itertools
import pickle
import matplotlib.pyplot as plt
import numpy as np
import math
import random
import os
import time
from numba import jit, njit
from numba.types import bool_, int_, float32
from math import comb
from copy import deepcopy
from tqdm import tqdm
import networkx as nx
import pandas as pd
from collections import defaultdict
from itertools import permutations

### Llegir fulls de càlcul 

In [ ]:
### Open the files
fulls = 8
#n_grupo = 4 - ara no estic fent servir grups

d = {} #crea un diccionari per emmagatzemar DataFrames(taules de memòria)
D = "Dataset" #prefix del nom de les fulles — al excel es diuen Dataset1, Dataset2, ...
for i in range(0,fulls): #recorro 0...7
    d["group" + str(i)] = pd.read_excel("datasets.ods", sheet_name=D+str(i+1)) #el que al diccionari serà gorup 0, sera el meu Dataset1
    # hem assignat al diccionari: d["group0"] = <taula Dataset1> 
    print('Read:',i, 'size', d["group" + str(i)].shape) #volem imprimir el nre de files i columnes

Read: 0 size (227, 183)
Read: 1 size (227, 183)
Read: 2 size (227, 183)
Read: 3 size (227, 183)
Read: 4 size (227, 183)
Read: 5 size (227, 183)
Read: 6 size (227, 183)
Read: 7 size (227, 183)


### Passar a matriu i binaritzem

Consideracions a tenir en compte. Primer, treiem les 3 primeres files i columnes de cada full de càlcul perquè a l'excel contenen etiquetes, no dades. En segon lloc, tenim 227 diles i 183 columnes. El que fem és forçar una matriu quadrada nre.filesxnre.files, de manera que, com la matriu inicial eren zeros, copio les columnes existets i deixo la resta plenes de zeros. Això podria donar error. NOTA: Els datasets compten la primera columna com títols.

In [8]:
## Hipermatrix M with the data (Only the synapses) 
rows = d["group" + str(i)].shape[0] - 3
columns = d["group" + str(i)].shape[1] - 3 #les tres primeres files i columnes no contenen dades

M = np.zeros((fulls,rows,columns))
for i in range(0, fulls):
    Data = d['group' + str(i)]
    M[i,:,:] = Data.iloc[3:,3:]
    
## Since we work with same number of nodes, we want them equal and square (zeros when no connections)
M_square = np.zeros((fulls, rows, rows))#creo matriu quadrada
M_square[:,:, 0:columns] = M[:,:,:] #copies les columnes existents (0..179) per deixar zeros en les restants (180..223).

## Binarization: No weights
#Lo queremos BINARIO, ignorando su peso (Luego pensar cómo se haría con el peso)
M_square_bin = np.zeros((fulls,rows,rows))
for i in range(fulls):
    for j in range(rows):
        for k in range(columns):
            if (M[i,j,k] >= 1):
                M_square_bin[i,j,k] = 1
                
Nx = rows # Number of nodes (we imposed rows == columns)


### Comprovació de l’estructura de la primera fulla

Vull assegurar-me de que la neteja dels Datasets (primeres 3 files i columnes) està feta correctament. Nota: Els Datasets prenen la primera fila com un Títol. Per tant, realment n'estic esborrant 4.

In [16]:
# inspecció ràpida del primer full de càlcul (group 0-->Dataset1)
df0 = d['group0']
print("Shape original group0:", df0.shape)
print("\nColumn names (first 12):\n", df0.columns[:12])
print("\nPrimeres 8 files i 8 columnes (sense transformar):")
print(df0.iloc[:8, :8])

Shape original group0: (227, 183)

Column names (first 12):
 Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Pre', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11'],
      dtype='object')

Primeres 8 files i 8 columnes (sense transformar):
  Unnamed: 0 Unnamed: 1 Unnamed: 2      Pre Unnamed: 4 Unnamed: 5 Unnamed: 6  \
0        NaN        NaN        NaN  Sensory        NaN        NaN        NaN   
1        NaN        NaN        NaN     ADFL       ADFR       ADLL       ADLR   
2        NaN        NaN        NaN      NaN        NaN        NaN        NaN   
3       Post    Sensory       ADFL        0          0          0          0   
4        NaN        NaN       ADFR        0          0          0          0   
5        NaN        NaN       ADLL        0          0          0          0   
6        NaN        NaN       ADLR        0          0          0          0   
7        NaN        NaN       AFDL      

## Escollir quin és el dataset A2

Sabem (per la informació del paper) que A2 té aproximadament 2186 arestes (edges), comprovem si el Dataset8 concorda amb aquestes dades, mirem cada dataset (recordem que group0-->Dataset1, group1-->Dataset2, ...).

NOTA: A més sospitem que el Dataset8 és el que conté les dades de A2 perquè al repo de la Teresa es pren A2 com a Blueprint fent M_square_bin[-1] que correspon a l'últim element de l'array, és a dir, M_square_bin[7], que correspon a group7--> Dataset8

### Coprovació de que Dataset8 correspon a A2

In [17]:
# Veure quants edges té cada dataset
for i in range(8):
    print(i, M_square_bin[i].sum())

0 775.0
1 986.0
2 1012.0
3 1136.0
4 1515.0
5 1525.0
6 2202.0
7 2186.0
